---

# ⚡ Dash for Energy Analytics — Mini Tutorial

Dash (by Plotly) lets you build **interactive web apps** in **pure Python**: no need to write HTML/JS. You define:

* a **layout** (what the page looks like),
* and **callbacks** (how UI → updates → figures/tables).

Perfect for dashboards like **demand monitoring**, **generation mix**, or **price vs demand**.

---

## Setup 🛠️

```bash
pip install dash plotly pandas numpy
# optional extras:
# pip install dash-bootstrap-components
```

Run a Dash app with:

```bash
python app.py
# then open http://127.0.0.1:8050 in your browser
```



---
# Examples 
## Smallest Working App (“Hello, Dash”) 👋

**app.py**

```python
from dash import Dash, html

app = Dash(__name__)
app.layout = html.Div([
    html.H1("Energy Dashboard"),
    html.P("Hello, Dash!")
])

if __name__ == "__main__":
    app.run_server(debug=True)
```

Open the URL in your browser. You should see a page with a title and text.

---



## Add a Plotly Figure (time series line) 📈

```python
from dash import Dash, html, dcc
import pandas as pd
import numpy as np
import plotly.express as px

# --- synthetic hourly demand (7 days)
rng = pd.date_range("2024-01-01", periods=24*7, freq="h")
demand = 1200 + 250*np.sin(2*np.pi*(rng.hour/24)) + np.random.normal(0, 40, len(rng))
df = pd.DataFrame({"time": rng, "demand_MW": demand})

app = Dash(__name__)
fig = px.line(df, x="time", y="demand_MW", title="Hourly Electricity Demand")

app.layout = html.Div([
    html.H2("Energy Demand (Interactive Plotly in Dash)"),
    dcc.Graph(figure=fig, id="demand-graph")
])

if __name__ == "__main__":
    app.run_server(debug=True)
```

Now the page shows an **interactive** Plotly chart (zoom, pan, hover).

---

## First Callback: Dropdown controls the plot 🎛️

We’ll add a **dropdown** to switch between **Week 1** and **Week 2** data and update the figure via a **callback**.

```python
from dash import Dash, html, dcc, Input, Output
import pandas as pd
import numpy as np
import plotly.express as px

# synthetic data for two weeks
rng = pd.date_range("2024-02-01", periods=24*7, freq="h")
week1 = 1100 + 240*np.sin(2*np.pi*(rng.hour/24)) + np.random.normal(0, 35, len(rng))
week2 = week1 * 1.07 + np.random.normal(0, 20, len(rng))

df1 = pd.DataFrame({"time": rng, "demand_MW": week1, "label": "Week 1"})
df2 = pd.DataFrame({"time": rng, "demand_MW": week2, "label": "Week 2"})
DATA = {"Week 1": df1, "Week 2": df2}

app = Dash(__name__)
app.layout = html.Div([
    html.H2("Demand Viewer"),
    dcc.Dropdown(
        id="week-dd",
        options=[{"label": k, "value": k} for k in DATA.keys()],
        value="Week 1", clearable=False, style={"width": 250}
    ),
    dcc.Graph(id="demand-graph")
])

@app.callback(
    Output("demand-graph", "figure"),
    Input("week-dd", "value")
)
def update_chart(week_key):
    df = DATA[week_key]
    fig = px.line(df, x="time", y="demand_MW", title=f"Hourly Demand — {week_key}")
    fig.update_layout(xaxis_title="Time", yaxis_title="Demand [MW]")
    return fig

if __name__ == "__main__":
    app.run_server(debug=True)
```

---


## Multiple Inputs: Temperature slider + weekday dropdown 🌡️📅

A classic demo: **demand vs temperature** with controls that change the data and re-draw the scatter.

```python
from dash import Dash, html, dcc, Input, Output
import plotly.express as px
import pandas as pd
import numpy as np

# synthetic demand vs temperature (with U-shape)
np.random.seed(0)
N = 800
temp   = np.random.uniform(-15, 30, N)
demand = 1250 + 5*(temp-15)**2 + np.random.normal(0, 60, N)
weekday = np.random.choice(["Weekday","Weekend"], size=N, p=[0.7, 0.3])
df = pd.DataFrame({"temp": temp, "demand": demand, "daytype": weekday})

app = Dash(__name__)
app.layout = html.Div([
    html.H2("Demand vs Temperature"),
    html.Div([
        html.Label("Min temperature (°C)"),
        html.Div(  # wrap Slider to style the container instead of the component
            dcc.Slider(
                id="tmin",
                min=-15, max=30, step=1, value=-10,
                marks=None,
                tooltip={"placement": "bottom", "always_visible": True},
                updatemode="mouseup"  # update on mouse release (smoother)
            ),
            style={"width": "400px", "marginRight": "30px"}
        ),
        html.Label("Day type"),
        html.Div(
            dcc.Dropdown(
                id="daytype",
                options=["All", "Weekday", "Weekend"],
                value="All",
                clearable=False
            ),
            style={"width": "200px"}
        ),
    ], style={"display": "flex", "alignItems": "center", "gap": "16px", "flexWrap": "wrap"}),

    dcc.Graph(id="scatter")
])

@app.callback(
    Output("scatter", "figure"),
    Input("tmin", "value"),
    Input("daytype", "value")
)
def filter_and_plot(tmin, daytype):
    sub = df[df["temp"] >= tmin]
    if daytype != "All":
        sub = sub[sub["daytype"] == daytype]
    fig = px.scatter(
        sub, x="temp", y="demand", color="daytype",
        title=f"Demand vs Temperature (temp ≥ {tmin}°C; {daytype})",
        labels={"temp": "Temperature (°C)", "demand": "Demand [MW]"}
    )
    return fig

if __name__ == "__main__":
    app.run_server(debug=True)

```

---



## Dual-Axis Demand vs Price (single figure) 📈💶

Dash doesn’t force multi-axis, that’s Plotly — but we can **drive** it with Dash controls later if we want.

```python
from dash import Dash, html, dcc
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pandas as pd
import numpy as np

idx = pd.date_range("2024-04-01", periods=30, freq="D")
dem = 1300 + 150*np.sin(2*np.pi*(np.arange(len(idx))/7)) + np.random.normal(0, 40, len(idx))
price = 50 + 0.07*(dem - dem.mean()) + np.random.normal(0, 4, len(idx))

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=idx, y=dem, name="Demand [MW]", mode="lines+markers"), secondary_y=False)
fig.add_trace(go.Bar(x=idx, y=price, name="Price [€/MWh]", opacity=0.5), secondary_y=True)
fig.update_layout(title_text="Demand vs Price", bargap=0.2, legend=dict(orientation="h", y=1.1))
fig.update_xaxes(title_text="Day")
fig.update_yaxes(title_text="Demand [MW]", secondary_y=False)
fig.update_yaxes(title_text="Price [€/MWh]", secondary_y=True)

app = Dash(__name__)
app.layout = html.Div([html.H2("Dual-Axis Example"), dcc.Graph(figure=fig)])

if __name__ == "__main__":
    app.run_server(debug=True)
```

---


## Date Picker + Live Subsetting of a Time Series 📆

```python
from dash import Dash, html, dcc, Input, Output
import pandas as pd
import numpy as np
import plotly.express as px

# 90 days hourly series
idx = pd.date_range("2024-01-01", periods=24*90, freq="h")
demand = 1200 + 260*np.sin(2*np.pi*(idx.hour/24)) + np.random.normal(0, 45, len(idx))
df = pd.DataFrame({"time": idx, "demand": demand})

app = Dash(__name__)
app.layout = html.Div([
    html.H2("Time Window Viewer"),
    dcc.DatePickerRange(
        id="range",
        start_date=idx.min().date(),
        end_date=idx.max().date(),
        display_format="YYYY-MM-DD"
    ),
    dcc.Graph(id="ts")
])

@app.callback(
    Output("ts", "figure"),
    Input("range", "start_date"),
    Input("range", "end_date")
)
def update_range(start_date, end_date):
    sub = df[(df["time"] >= start_date) & (df["time"] <= (pd.to_datetime(end_date) + pd.Timedelta(days=1)))]
    fig = px.line(sub, x="time", y="demand", title=f"Demand: {start_date} → {end_date}")
    fig.update_layout(xaxis_title="Time", yaxis_title="Demand [MW]")
    return fig

if __name__ == "__main__":
    app.run_server(debug=True)
```

---



## Periodic Auto-Refresh (simulate streaming) ⏱️

Use `dcc.Interval` to refresh data/plots (e.g., every 5 seconds).

```python
from dash import Dash, html, dcc, Input, Output
import plotly.express as px
import pandas as pd
import numpy as np
from datetime import datetime

app = Dash(__name__)
app.layout = html.Div([
    html.H2("Live Demand (simulated)"),
    dcc.Graph(id="live"),
    dcc.Interval(id="tick", interval=5000, n_intervals=0)  # 5 seconds
])

# keep a small rolling buffer in memory
BUF = pd.DataFrame(columns=["time","demand"])

@app.callback(
    Output("live", "figure"),
    Input("tick", "n_intervals")
)
def refresh(_):
    global BUF
    now = pd.Timestamp(datetime.now().replace(microsecond=0))
    new_row = pd.DataFrame({"time":[now], "demand":[1200 + np.random.normal(0,40)]})
    BUF = pd.concat([BUF, new_row], ignore_index=True).tail(100)  # last 100 points
    fig = px.line(BUF, x="time", y="demand", title="Live Demand (last 100 points)")
    fig.update_layout(xaxis_title="Time", yaxis_title="Demand [MW]")
    return fig

if __name__ == "__main__":
    app.run_server(debug=True)
```

---




## Simple Multi-Page Skeleton 🗂️

Two pages: **“Demand”** and **“Mix”**. Use the URL path to switch.

```python
from dash import Dash, html, dcc, Input, Output
import plotly.express as px
import pandas as pd
import numpy as np

app = Dash(__name__, suppress_callback_exceptions=True)
app.layout = html.Div([
    dcc.Location(id="url"),
    html.Div([
        dcc.Link("Demand", href="/"),
        html.Span(" | "),
        dcc.Link("Mix", href="/mix")
    ], style={"marginBottom": 12}),
    html.Div(id="page-content")
])

def demand_layout():
    rng = pd.date_range("2024-01-01", periods=24*7, freq="h")
    demand = 1200 + 240*np.sin(2*np.pi*(rng.hour/24)) + np.random.normal(0, 40, len(rng))
    fig = px.line(pd.DataFrame({"time": rng, "demand": demand}), x="time", y="demand", title="Demand (Week)")
    return html.Div([html.H3("Demand Page"), dcc.Graph(figure=fig)])

def mix_layout():
    days = pd.date_range("2024-03-01", periods=10, freq="D")
    solar = np.clip(200 + 50*np.sin(2*np.pi*(days.dayofyear/365)), 150, 300)
    wind  = 300 + 80*np.sin(2*np.pi*(days.dayofyear/14) + 1)
    hydro = 500 + np.random.normal(0, 25, len(days))
    long = (pd.DataFrame({"day": days, "Solar": solar, "Wind": wind, "Hydro": hydro})
            .melt(id_vars="day", var_name="source", value_name="MWh"))
    fig = px.area(long, x="day", y="MWh", color="source", title="Generation Mix (10 days)")
    return html.Div([html.H3("Mix Page"), dcc.Graph(figure=fig)])

@app.callback(Output("page-content", "children"), Input("url", "pathname"))
def route(path):
    if path == "/mix":
        return mix_layout()
    else:
        return demand_layout()

if __name__ == "__main__":
    app.run_server(debug=True)
```

---


# Home Energy Mini-Dashboard 🗂️

```python

from dash import Dash, html, dcc, Input, Output
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import pandas as pd

# ---------- Synthetic day of data (hourly) ----------
rng = pd.date_range("2024-05-01", periods=24, freq="h")
hours = np.arange(24)

# Demand profile (evening peak)
demand = 1.8 + 0.4*np.exp(-0.5*((hours-8)/2.3)**2) + 1.1*np.exp(-0.5*((hours-19)/2.6)**2)
demand += np.random.default_rng(0).normal(0, 0.05, size=24)   # noise
demand = np.clip(demand, 0, None)  # kW

# Forecast (noisy version of demand, as if day-ahead)
forecast = demand + np.random.default_rng(1).normal(0, 0.1, size=24)

# Simple PV bell curve (kW)
pv = 0.0 + 2.8*np.exp(-0.5*((hours-13)/3.0)**2)  # midday peak ~2.8 kW

df_base = pd.DataFrame({"time": rng, "demand_kW": demand, "forecast_kW": forecast, "pv_kW": pv}).set_index("time")

# ---------- Battery simulator ----------
def simulate_battery(df, cap_kwh=5.0, eta=0.95, soc0_ratio=0.5, dt_h=1.0):
    """
    Very simple self-consumption battery:
    - charge from PV surplus (pv > demand), up to capacity
    - discharge to cover demand (or forecast) when pv < demand
    - unlimited power rate for simplicity (educational)
    - round-trip modeled as symmetric efficiency factor on charge+discharge
    """
    demand = df["demand_kW"].values.copy()
    pv = df["pv_kW"].values.copy()
    n = len(demand)

    soc = np.zeros(n+1)
    soc[0] = soc0_ratio * cap_kwh  # initial SoC in kWh
    grid = np.zeros(n)             # net grid import (+ import, negative means export)
    ch = np.zeros(n)               # battery charge power (kW)
    dis = np.zeros(n)              # battery discharge power (kW)

    for t in range(n):
        load = demand[t]
        gen  = pv[t]
        net  = load - gen  # positive means net load; negative means surplus PV

        if net > 0:
            # Need energy -> try to discharge battery
            # available discharge energy this hour (kWh) respecting SoC and efficiency
            e_need = net * dt_h                      # kWh
            e_avail = soc[t] * eta                   # usable energy considering efficiency
            e_use = min(e_need, e_avail)
            dis[t] = e_use / dt_h                    # kW
            soc[t+1] = soc[t] - e_use/eta            # battery loses e_use/eta from SoC
            grid[t] = net - dis[t]                   # remaining from grid
        else:
            # Surplus PV -> try to charge battery
            e_surplus = -net * dt_h                  # kWh
            e_room = (cap_kwh - soc[t])              # kWh free
            e_store = min(e_surplus * eta, e_room)   # store with charge efficiency
            ch[t] = e_store / dt_h                   # kW
            soc[t+1] = soc[t] + e_store              # SoC increases by e_store
            grid[t] = net + ch[t]                    # net after charging (often still <= 0)

    out = df.copy()
    out["grid_kW"] = grid
    out["bat_charge_kW"] = ch
    out["bat_discharge_kW"] = dis
    out["soc_kWh"] = soc[1:]
    return out

def kpis(df_sim):
    # Energy terms (kWh) with 1h timestep
    load = df_sim["demand_kW"].sum()
    pv = df_sim["pv_kW"].sum()
    grid_import = df_sim["grid_kW"].clip(lower=0).sum()
    grid_export = (-df_sim["grid_kW"].clip(upper=0)).sum()
    self_consumed = pv - grid_export
    self_consumption_ratio = self_consumed / pv if pv > 0 else 0.0
    peak_grid = df_sim["grid_kW"].max()
    return dict(
        load_kWh=load,
        pv_kWh=pv,
        grid_import_kWh=grid_import,
        grid_export_kWh=grid_export,
        self_consumption_pct=100* self_consumption_ratio,
        peak_grid_kW=peak_grid
    )

# ---------- Dash app ----------
app = Dash(__name__)
app.title = "HEMS Mini"

app.layout = html.Div([
    html.H2("Home Energy Mini-Dashboard"),
    html.P("Adjust battery capacity to see how grid import and self-consumption change."),

    html.Div([
        html.Div([
            html.Label("Battery capacity (kWh)"),
            dcc.Slider(
                id="cap",
                min=0, max=20, step=1, value=5,
                marks={0:"0", 5:"5", 10:"10", 15:"15", 20:"20"},
                tooltip={"placement": "bottom", "always_visible": True}
            ),
        ], style={"width": "420px", "marginRight": "24px"}),

        html.Div([
            html.Label("Round-trip efficiency (0.80–1.00)"),
            dcc.Slider(
                id="eta",
                min=0.80, max=1.00, step=0.01, value=0.95,
                marks={0.80:"0.80", 0.9:"0.90", 1.0:"1.00"},
                tooltip={"placement": "bottom", "always_visible": False}
            ),
        ], style={"width": "420px", "marginRight": "24px"}),

        html.Div([
            html.Label("Initial SoC (%)"),
            dcc.Slider(
                id="soc0",
                min=0, max=100, step=5, value=50,
                marks={0:"0", 50:"50", 100:"100"},
                tooltip={"placement": "bottom", "always_visible": False}
            ),
        ], style={"width": "420px"}),
    ], style={"display": "flex", "flexWrap": "wrap", "alignItems": "center", "gap": "16px"}),

    dcc.Graph(id="timeseries", style={"height": "420px", "marginTop": "10px"}),

    html.Div(id="kpi", style={"display": "flex", "gap": "24px", "flexWrap": "wrap", "marginTop": "8px"})
], style={"maxWidth": "1200px", "margin": "0 auto", "fontFamily": "sans-serif"})

@app.callback(
    Output("timeseries", "figure"),
    Output("kpi", "children"),
    Input("cap", "value"),
    Input("eta", "value"),
    Input("soc0", "value"),
)
def update(cap, eta, soc0):
    df_sim = simulate_battery(df_base, cap_kwh=float(cap), eta=float(eta), soc0_ratio=float(soc0)/100.0)

    # KPIs
    m = kpis(df_sim)
    kpi_boxes = [
        html.Div([
            html.H4("Load (kWh)"),
            html.P(f"{m['load_kWh']:.1f}")
        ], style=box_style()),
        html.Div([
            html.H4("PV (kWh)"),
            html.P(f"{m['pv_kWh']:.1f}")
        ], style=box_style()),
        html.Div([
            html.H4("Grid import (kWh)"),
            html.P(f"{m['grid_import_kWh']:.1f}")
        ], style=box_style()),
        html.Div([
            html.H4("Grid export (kWh)"),
            html.P(f"{m['grid_export_kWh']:.1f}")
        ], style=box_style()),
        html.Div([
            html.H4("Self-consumption (%)"),
            html.P(f"{m['self_consumption_pct']:.1f}%")
        ], style=box_style()),
        html.Div([
            html.H4("Peak grid (kW)"),
            html.P(f"{m['peak_grid_kW']:.2f}")
        ], style=box_style()),
    ]

    # Figure
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df_sim.index, y=df_sim["demand_kW"], name="Demand [kW]", line=dict(color="#1f77b4")))
    fig.add_trace(go.Scatter(x=df_sim.index, y=df_sim["forecast_kW"], name="Forecast [kW]", line=dict(color="#ff7f0e", dash="dot")))
    fig.add_trace(go.Scatter(x=df_sim.index, y=df_sim["pv_kW"], name="PV [kW]", line=dict(color="#2ca02c")))
    fig.add_trace(go.Scatter(x=df_sim.index, y=df_sim["grid_kW"], name="Grid after battery [kW]", line=dict(color="#d62728")))
    fig.add_trace(go.Scatter(x=df_sim.index, y=df_sim["bat_charge_kW"], name="Battery charge [kW]", line=dict(color="#9467bd"), visible="legendonly"))
    fig.add_trace(go.Scatter(x=df_sim.index, y=df_sim["bat_discharge_kW"], name="Battery discharge [kW]", line=dict(color="#8c564b"), visible="legendonly"))
    fig.add_trace(go.Scatter(x=df_sim.index, y=df_sim["soc_kWh"], name="SoC [kWh]", line=dict(color="#17becf"), yaxis="y2"))

    fig.update_layout(
        title=f"HEMS Simulation — Battery {cap} kWh, η={eta:.2f}, SoC0={soc0}%",
        xaxis_title="Time",
        yaxis_title="Power [kW]",
        legend=dict(orientation="h", y=0.95, x=1, xanchor="right", yanchor="bottom"),
        margin=dict(l=40, r=40, t=80, b=40),
        template="plotly_white",
        yaxis2=dict(
            title="State of Charge [kWh]",
            overlaying="y",
            side="right",
            showgrid=False
        ),
    )
    return fig, kpi_boxes

def box_style():
    return {
        "background": "#f7f7f9",
        "padding": "10px 14px",
        "border": "1px solid #e6e6e6",
        "borderRadius": "10px",
        "minWidth": "160px",
    }

if __name__ == "__main__":
    app.run_server(debug=True)
```

# Deployment notes 🚀

### Local
`python app.py`, open `http://127.0.0.1:8050`.

Here are the *realistic* **free (or effectively free)** ways to host a Plotly **Dash** app — plus tiny setups for each and how to embed the app back into your GitHub Pages/Jupyter Book site.

### ✅ Best free options 

1. **Render – Free Web Service**
   Always-on *free* web services (with limits). Good default for Dash. Deploy from your GitHub repo; sleeps less than most hobby hosts. 

2. **Hugging Face Spaces – Free via Docker**
   Spaces can run **Dash** apps using a Dockerfile; great for demos, public links, easy embeds. 

3. **Fly.io – Free allowances**
   Enough monthly free resources to run a tiny app if you stay within the free allowances; needs a lightweight VM + simple Docker. 

4. **PythonAnywhere – Free plan (limited)**
   One free web app, outbound internet restrictions, lower resources; OK for simple Dash apps. 

> **Railway** moved to usage credits + paid plans; not a stable “free forever” option anymore. 
